<a name="top"></a><img src="images/chisel_1024.png" alt="Chisel logo" style="width:480px;" />

# Module 3 Interlude: Chisel Standard Library
**Prev: [Generators: Collections](3.2_collections.ipynb)**<br>
**Next: [Higher-Order Functions](3.3_higher-order_functions.ipynb)**

## Motivation
Chisel은 모두 재사용에 관한 것이므로 일반적으로 사용되는 하드웨어 블록에 대한 인터페이스 및 생성기(RTL의 상호 운용성 장려)의 표준 라이브러리를 제공하는 것이 합리적입니다.

## Setup

In [1]:
val path = System.getProperty("user.dir") + "/source/load-ivy.sc"
interp.load.module(ammonite.ops.Path(java.nio.file.FileSystems.getDefault().getPath(path)))

path: String = "/home/parkdongho/dev/chisel-bootcamp-kr/source/load-ivy.sc"

In [2]:
import chisel3._
import chisel3.util._
import chisel3.tester._
import chisel3.tester.RawTester.test

import chisel3._

import chisel3.util._

import chisel3.tester._

import chisel3.tester.RawTester.test

---
# The Cheatsheet
[Chisel3 치트시트](https://github.com/freechipsproject/chisel-cheatsheet/releases/latest/download/chisel_cheatsheet.pdf)에는 우리가 제공하는 일부 표준 라이브러리 유틸리티를 포함하여 모든 주요 하드웨어 구성 API에 대한 요약이 포함되어 있습니다. 아래에서 소개합니다.

# Decoupled: A Standard Ready-Valid Interface
Chisel에서 제공하는 일반적으로 사용되는 인터페이스 중 하나는 `DecoupledIO`로, 데이터 전송을 위해 즉시 사용할 수 있는 인터페이스를 제공합니다. 아이디어는 소스가 전송할 데이터로 `bits` 신호를 구동하고 전송할 데이터가 있을 때 `valid` 신호를 구동한다는 것입니다. 싱크는 데이터를 받아들일 준비가 되었을 때 `ready` 신호를 구동하고, 주기에서 `ready`와 `valid`가 모두 선언되면 데이터가 전송된 것으로 간주됩니다.

이것은 backpressure 메커니즘을 포함하여 데이터 전송을 위해 양방향으로 flow control mechanism을 제공합니다.

참고: `ready`와 `valid`는 combinational하게 연결하면 안 됩니다. 그렇지 않으면 합성할 수 없는 combinational loop가 발생할 수 있습니다. `ready`는 싱크가 데이터를 수신할 수 있는지 여부에만 의존해야 하고 `valid`는 소스에 데이터가 있는지 여부에만 의존해야 합니다. 트랜잭션 후에(다음 클록 주기에서) 값이 업데이트되어야 합니다.

모든 Chisel 데이터는 다음과 같이 `DecoupledIO`(`bits` 필드로 사용됨)로 래핑될 수 있습니다.

```scala
val myChiselData = UInt(8.W)
// or any Chisel data type, such as Bool(), SInt(...), or even custom Bundles
val myDecoupled = Decoupled(myChiselData)
```

위는 필드가 있는 새로운 'DecoupledIO' 번들을 생성합니다.
- `valid`: Output(Bool)
- `ready`: Input(Bool)
- `bits`: Output(UInt(8.W))
___

나머지 섹션은 이전과 약간 다르게 구성됩니다. 코딩 연습을 제공하는 대신 회로 상태를 인쇄하는 몇 가지 코드 예제와 테스트 케이스를 제공할 것입니다. 테스트를 실행하기 전에 인쇄될 내용을 예측해 보십시오.

## Queues

`Queue`는 양쪽에 Decoupled interfaces가 있는 FIFO(first-in, first-out) queue를 생성하여 backpressure을 허용합니다. 데이터 타입과 elements 수를 모두 구성 가능합니다.

> 처음 몇 번은 `litValue`라고 부르는 `peek` 값을 인쇄하는 방법에 주목하세요. 이것은 반환되는 Chisel 리터럴을 `BigInt`로 변환합니다. 나중에 유사한 호출에서는 `litValue`를 호출하지 않으며 `peek`이 반환하는 값에 대한 추가 정보를 볼 수 있습니다. 유형과 너비처럼.

In [3]:
test(new Module {
    // Example circuit using a Queue
    val io = IO(new Bundle {
      val in = Flipped(Decoupled(UInt(8.W)))
      val out = Decoupled(UInt(8.W))
    })
    val queue = Queue(io.in, 2)  // 2-element queue
    io.out <> queue
  }) { c =>
    c.io.out.ready.poke(false.B)
    c.io.in.valid.poke(true.B)  // Enqueue an element
    c.io.in.bits.poke(42.U)
    println(s"Starting:")
    println(s"\tio.in: ready=${c.io.in.ready.peek().litValue}")
    println(s"\tio.out: valid=${c.io.out.valid.peek().litValue}, bits=${c.io.out.bits.peek().litValue}")
    c.clock.step(1)

    c.io.in.valid.poke(true.B)  // Enqueue another element
    c.io.in.bits.poke(43.U)
    // What do you think io.out.valid and io.out.bits will be?
    println(s"After first enqueue:")
    println(s"\tio.in: ready=${c.io.in.ready.peek().litValue}")
    println(s"\tio.out: valid=${c.io.out.valid.peek().litValue}, bits=${c.io.out.bits.peek().litValue}")
    c.clock.step(1)

    c.io.in.valid.poke(true.B)  // Read a element, attempt to enqueue
    c.io.in.bits.poke(44.U)
    c.io.out.ready.poke(true.B)
    // What do you think io.in.ready will be, and will this enqueue succeed, and what will be read?
    println(s"On first read:")
    println(s"\tio.in: ready=${c.io.in.ready.peek()}")
    println(s"\tio.out: valid=${c.io.out.valid.peek()}, bits=${c.io.out.bits.peek()}")
    c.clock.step(1)

    c.io.in.valid.poke(false.B)  // Read elements out
    c.io.out.ready.poke(true.B)
    // What do you think will be read here?
    println(s"On second read:")
    println(s"\tio.in: ready=${c.io.in.ready.peek()}")
    println(s"\tio.out: valid=${c.io.out.valid.peek()}, bits=${c.io.out.bits.peek()}")
    c.clock.step(1)

    // Will a third read produce anything?
    println(s"On third read:")
    println(s"\tio.in: ready=${c.io.in.ready.peek()}")
    println(s"\tio.out: valid=${c.io.out.valid.peek()}, bits=${c.io.out.bits.peek()}")
    c.clock.step(1)
}

Elaborating design...
Done elaborating.
Starting:
	io.in: ready=1
	io.out: valid=0, bits=0
After first enqueue:
	io.in: ready=1
	io.out: valid=1, bits=42
On first read:
	io.in: ready=Bool(false)
	io.out: valid=Bool(true), bits=UInt<8>(42)
On second read:
	io.in: ready=Bool(true)
	io.out: valid=Bool(true), bits=UInt<8>(43)
On third read:
	io.in: ready=Bool(true)
	io.out: valid=Bool(false), bits=UInt<8>(42)
test Helper_Anon Success: 0 tests passed in 7 cycles in 0.028501 seconds 245.60 Hz


## Arbiters
중재자는 우선 순위가 지정된 _n_ `DecoupledIO` 소스에서 하나의 `DecoupledIO` 싱크로 데이터를 라우팅합니다.
치즐에는 두 가지 유형이 포함되어 있습니다.
- `Arbiter`: 지수가 낮은 생산자를 우선시합니다.
- `RRArbiter`: 라운드 로빈 순서로 실행

Arbiter 라우팅은 조합 논리로 구현됩니다.

아래 예는 우선순위 중재자의 사용을 보여줍니다(다음 섹션에서도 구현할 것입니다).

In [4]:
test(new Module {
    // Example circuit using a priority arbiter
    val io = IO(new Bundle {
      val in = Flipped(Vec(2, Decoupled(UInt(8.W))))
      val out = Decoupled(UInt(8.W))
    })
    // Arbiter doesn't have a convenience constructor, so it's built like any Module
    val arbiter = Module(new Arbiter(UInt(8.W), 2))  // 2 to 1 Priority Arbiter
    arbiter.io.in <> io.in
    io.out <> arbiter.io.out
  }) { c =>
    c.io.in(0).valid.poke(false.B)
    c.io.in(1).valid.poke(false.B)
    c.io.out.ready.poke(false.B)
    println(s"Start:")
    println(s"\tin(0).ready=${c.io.in(0).ready.peek().litValue}, in(1).ready=${c.io.in(1).ready.peek().litValue}")
    println(s"\tout.valid=${c.io.out.valid.peek().litValue}, out.bits=${c.io.out.bits.peek().litValue}")
    c.io.in(1).valid.poke(true.B)  // Valid input 1
    c.io.in(1).bits.poke(42.U)
    c.io.out.ready.poke(true.B)
    // What do you think the output will be?
    println(s"valid input 1:")
    println(s"\tin(0).ready=${c.io.in(0).ready.peek().litValue}, in(1).ready=${c.io.in(1).ready.peek().litValue}")
    println(s"\tout.valid=${c.io.out.valid.peek().litValue}, out.bits=${c.io.out.bits.peek().litValue}")
    c.io.in(0).valid.poke(true.B)  // Valid inputs 0 and 1
    c.io.in(0).bits.poke(43.U)
    // What do you think the output will be? Which inputs will be ready?
    println(s"valid inputs 0 and 1:")
    println(s"\tin(0).ready=${c.io.in(0).ready.peek().litValue}, in(1).ready=${c.io.in(1).ready.peek().litValue}")
    println(s"\tout.valid=${c.io.out.valid.peek().litValue}, out.bits=${c.io.out.bits.peek().litValue}")
    c.io.in(1).valid.poke(false.B)  // Valid input 0
    // What do you think the output will be?
    println(s"valid input 0:")
    println(s"\tin(0).ready=${c.io.in(0).ready.peek().litValue}, in(1).ready=${c.io.in(1).ready.peek().litValue}")
    println(s"\tout.valid=${c.io.out.valid.peek().litValue}, out.bits=${c.io.out.bits.peek().litValue}")
}

Elaborating design...
Done elaborating.
Start:
	in(0).ready=0, in(1).ready=0
	out.valid=0, out.bits=0
valid input 1:
	in(0).ready=1, in(1).ready=1
	out.valid=1, out.bits=42
valid inputs 0 and 1:
	in(0).ready=1, in(1).ready=0
	out.valid=1, out.bits=43
valid input 0:
	in(0).ready=1, in(1).ready=0
	out.valid=1, out.bits=43
test Helper_Anon Success: 0 tests passed in 2 cycles in 0.006201 seconds 322.55 Hz


# Misc Function Blocks
Chisel Utils에는 stateless 기능을 수행하는 몇 가지 도우미가 있습니다.

## Bitwise Utilities
### PopCount
PopCount는 입력의 high (1) 비트 수를 `UInt`로 반환합니다.

### Reverse
Reverse는 비트 반전 입력을 반환합니다.

In [5]:
test(new Module {
    // Example circuit using PopCount
    val io = IO(new Bundle {
      val in = Input(UInt(8.W))
      val out = Output(UInt(8.W))
    })
    io.out := PopCount(io.in)
  }) { c =>
    // Integer.parseInt is used create an Integer from a binary specification
    c.io.in.poke(Integer.parseInt("00000000", 2).U)
    println(s"in=0b${c.io.in.peek().litValue.toInt.toBinaryString}, out=${c.io.out.peek().litValue}")

    c.io.in.poke(Integer.parseInt("00001111", 2).U)
    println(s"in=0b${c.io.in.peek().litValue.toInt.toBinaryString}, out=${c.io.out.peek().litValue}")

    c.io.in.poke(Integer.parseInt("11001010", 2).U)
    println(s"in=0b${c.io.in.peek().litValue.toInt.toBinaryString}, out=${c.io.out.peek().litValue}")

    c.io.in.poke(Integer.parseInt("11111111", 2).U)
    println(s"in=0b${c.io.in.peek().litValue.toInt.toBinaryString}, out=${c.io.out.peek().litValue}")

}

Elaborating design...
Done elaborating.
in=0b0, out=0
in=0b1111, out=4
in=0b11001010, out=4
in=0b11111111, out=8
test Helper_Anon Success: 0 tests passed in 2 cycles in 0.003028 seconds 660.59 Hz


In [6]:
test(new Module {
    // Example circuit using Reverse
    val io = IO(new Bundle {
      val in = Input(UInt(8.W))
      val out = Output(UInt(8.W))
    })
    io.out := Reverse(io.in)
  }) { c =>
    // Integer.parseInt is used create an Integer from a binary specification
    c.io.in.poke(Integer.parseInt("01010101", 2).U)
    println(s"in=0b${c.io.in.peek().litValue.toInt.toBinaryString}, out=0b${c.io.out.peek().litValue.toInt.toBinaryString}")

    c.io.in.poke(Integer.parseInt("00001111", 2).U)
    println(s"in=0b${c.io.in.peek().litValue.toInt.toBinaryString}, out=0b${c.io.out.peek().litValue.toInt.toBinaryString}")

    c.io.in.poke(Integer.parseInt("11110000", 2).U)
    println(s"in=0b${c.io.in.peek().litValue.toInt.toBinaryString}, out=0b${c.io.out.peek().litValue.toInt.toBinaryString}")

    c.io.in.poke(Integer.parseInt("11001010", 2).U)
    println(s"in=0b${c.io.in.peek().litValue.toInt.toBinaryString}, out=0b${c.io.out.peek().litValue.toInt.toBinaryString}")
}

Elaborating design...
Done elaborating.
in=0b1010101, out=0b10101010
in=0b1111, out=0b11110000
in=0b11110000, out=0b1111
in=0b11001010, out=0b1010011
test Helper_Anon Success: 0 tests passed in 2 cycles in 0.002855 seconds 700.62 Hz


## OneHot encoding utilities
OneHot은 각 값에 대해 하나의 와이어가 있고 정확히 하나의 와이어만 high인 정수 인코딩입니다. 이를 통해 mux와 같은 일부 기능을 효율적으로 생성할 수 있습니다. 그러나 1와이어 하이 조건이 유지되지 않으면 동작이 정의되지 않을 수 있습니다.

아래 두 함수는 바이너리(`UInt`)와 OneHot 인코딩 간의 변환을 제공하며 서로 반대입니다.
- UInt to OneHot: `UIntToOH`
- OneHot to UInt: `OHToUInt`

In [7]:
test(new Module {
    // Example circuit using UIntToOH
    val io = IO(new Bundle {
      val in = Input(UInt(4.W))
      val out = Output(UInt(16.W))
    })
    io.out := UIntToOH(io.in)
  }) { c =>
    c.io.in.poke(0.U)
    println(s"in=${c.io.in.peek().litValue}, out=0b${c.io.out.peek().litValue.toInt.toBinaryString}")

    c.io.in.poke(1.U)
    println(s"in=${c.io.in.peek().litValue}, out=0b${c.io.out.peek().litValue.toInt.toBinaryString}")

    c.io.in.poke(8.U)
    println(s"in=${c.io.in.peek().litValue}, out=0b${c.io.out.peek().litValue.toInt.toBinaryString}")

    c.io.in.poke(15.U)
    println(s"in=${c.io.in.peek().litValue}, out=0b${c.io.out.peek().litValue.toInt.toBinaryString}")
}

Elaborating design...
Done elaborating.
in=0, out=0b1
in=1, out=0b10
in=8, out=0b100000000
in=15, out=0b1000000000000000
test Helper_Anon Success: 0 tests passed in 2 cycles in 0.002347 seconds 852.07 Hz


In [8]:
test(new Module {
    // Example circuit using OHToUInt
    val io = IO(new Bundle {
      val in = Input(UInt(16.W))
      val out = Output(UInt(4.W))
    })
    io.out := OHToUInt(io.in)
}) { c =>
    c.io.in.poke(Integer.parseInt("0000 0000 0000 0001".replace(" ", ""), 2).U)
    println(s"in=0b${c.io.in.peek().litValue.toInt.toBinaryString}, out=${c.io.out.peek().litValue}")

    c.io.in.poke(Integer.parseInt("0000 0000 1000 0000".replace(" ", ""), 2).U)
    println(s"in=0b${c.io.in.peek().litValue.toInt.toBinaryString}, out=${c.io.out.peek().litValue}")

    c.io.in.poke(Integer.parseInt("1000 0000 0000 0001".replace(" ", ""), 2).U)
    println(s"in=0b${c.io.in.peek().litValue.toInt.toBinaryString}, out=${c.io.out.peek().litValue}")

    // Some invalid inputs:
    // None high
    c.io.in.poke(Integer.parseInt("0000 0000 0000 0000".replace(" ", ""), 2).U)
    println(s"in=0b${c.io.in.peek().litValue.toInt.toBinaryString}, out=${c.io.out.peek().litValue}")

    // Multiple high
    c.io.in.poke(Integer.parseInt("0001 0100 0010 0000".replace(" ", ""), 2).U)
    println(s"in=0b${c.io.in.peek().litValue.toInt.toBinaryString}, out=${c.io.out.peek().litValue}")
}


Elaborating design...
Done elaborating.
in=0b1, out=0
in=0b10000000, out=7
in=0b1000000000000001, out=15
in=0b0, out=0
in=0b1010000100000, out=15
test Helper_Anon Success: 0 tests passed in 2 cycles in 0.002834 seconds 705.60 Hz


## Muxes
이러한 mux는 선택 신호가 있는 값 목록을 가져와 가장 낮은 인덱스 선택 신호와 관련된 값을 출력합니다.

이것들은 (select: Bool, value: Data) 튜플의 목록 또는 해당하는 선택 및 값 목록을 인수로 사용할 수 있습니다. 간단하게 하기 위해 아래 예에서는 두 번째 형식만 보여줍니다.

### Priority Mux
`PriorityMux`는 가장 낮은 인덱스 어설션 선택 신호와 관련된 값을 출력합니다.

### OneHot Mux
`Mux1H`는 선택 신호 중 정확히 하나가 높음이 보장될 때 효율적인 구현을 제공합니다. 가정이 사실이 아니면 동작이 정의되지 않습니다.

In [7]:
test(new Module {
    // Example circuit using PriorityMux
    val io = IO(new Bundle {
      val in_sels = Input(Vec(2, Bool()))
      val in_bits = Input(Vec(2, UInt(8.W)))
      val out = Output(UInt(8.W))
    })
    io.out := PriorityMux(io.in_sels, io.in_bits)
  }) { c =>
    c.io.in_bits(0).poke(10.U)
    c.io.in_bits(1).poke(20.U)

    // Select higher index only
    c.io.in_sels(0).poke(false.B)
    c.io.in_sels(1).poke(true.B)
    println(s"in_sels=${c.io.in_sels(0).peek().litValue}, out=${c.io.out.peek().litValue}")

    // Select both - arbitration needed
    c.io.in_sels(0).poke(true.B)
    c.io.in_sels(1).poke(true.B)
    println(s"in_sels=${c.io.in_sels(0).peek().litValue}, out=${c.io.out.peek().litValue}")

    // Select lower index only
    c.io.in_sels(0).poke(true.B)
    c.io.in_sels(1).poke(false.B)
    println(s"in_sels=${c.io.in_sels(0).peek().litValue}, out=${c.io.out.peek().litValue}")
}

Elaborating design...
Done elaborating.
in_sels=0, out=20
in_sels=1, out=10
in_sels=1, out=10
test Helper_Anon Success: 0 tests passed in 2 cycles in 0.003274 seconds 610.89 Hz


In [12]:
test(new Module {
    // Example circuit using Mux1H
    val io = IO(new Bundle {
      val in_sels = Input(Vec(2, Bool()))
      val in_bits = Input(Vec(2, UInt(8.W)))
      val out = Output(UInt(8.W))
    })
    io.out := Mux1H(io.in_sels, io.in_bits)
  }) { c =>
    c.io.in_bits(0).poke(10.U)
    c.io.in_bits(1).poke(20.U)

    // Select index 1
    c.io.in_sels(0).poke(false.B)
    c.io.in_sels(1).poke(true.B)
    println(s"in_sels=${c.io.in_sels(0).peek().litValue}${c.io.in_sels(1).peek().litValue}, out=${c.io.out.peek().litValue}")

    // Select index 0
    c.io.in_sels(0).poke(true.B)
    c.io.in_sels(1).poke(false.B)
    println(s"in_sels=${c.io.in_sels(0).peek().litValue}${c.io.in_sels(1).peek().litValue}, out=${c.io.out.peek().litValue}")

    // Select none (invalid)
    c.io.in_sels(0).poke(false.B)
    c.io.in_sels(1).poke(false.B)
    println(s"in_sels=${c.io.in_sels(0).peek().litValue}${c.io.in_sels(1).peek().litValue}, out=${c.io.out.peek().litValue}")

    // Select both (invalid)
    c.io.in_sels(0).poke(true.B)
    c.io.in_sels(1).poke(true.B)
    println(s"in_sels=${c.io.in_sels(0).peek().litValue}, out=${c.io.out.peek().litValue}")
}

Elaborating design...
Done elaborating.
in_sels=01, out=20
in_sels=10, out=10
in_sels=00, out=0
in_sels=1, out=30
test Helper_Anon Success: 0 tests passed in 2 cycles in 0.003496 seconds 572.16 Hz


## Counter
`Counter`는 지정된 한도까지 사이클마다 한 번씩 증가할 수 있는 카운터이며, 이 지점에서 오버플로가 발생합니다. 모듈이 **아닙니다**. 값에 액세스할 수 있습니다.

In [11]:
test(new Module {
    // Example circuit using Mux1H
    val io = IO(new Bundle {
      val count = Input(Bool())
      val out = Output(UInt(2.W))
    })
    val counter = Counter(3)  // 3-count Counter (outputs range [0...2])
    when(io.count) {
      counter.inc()
    }
    io.out := counter.value
  }) { c =>
    c.io.count.poke(true.B)
    println(s"start: counter value=${c.io.out.peek().litValue}")

    c.clock.step(1)
    println(s"step 1: counter value=${c.io.out.peek().litValue}")

    c.clock.step(1)
    println(s"step 2: counter value=${c.io.out.peek().litValue}")

    c.io.count.poke(false.B)
    c.clock.step(1)
    println(s"step without increment: counter value=${c.io.out.peek().litValue}")

    c.io.count.poke(true.B)
    c.clock.step(1)
    println(s"step again: counter value=${c.io.out.peek().litValue}")
}

Elaborating design...
Done elaborating.
start: counter value=0
step 1: counter value=1
step 2: counter value=2
step without increment: counter value=2
step again: counter value=0
test Helper_Anon Success: 0 tests passed in 6 cycles in 0.002886 seconds 2079.01 Hz


---
# You're done!

[Return to the top.](#top)